In [1]:
import numpy as np
import pandas as pd

In [2]:
# read in csv file
df = pd.read_csv('coorteeqsrafva.csv', sep=';', header=0, index_col=0)
df.head()

,diagnosi,ecg_id,ritmi,patient_id,age,sex,height,weight,nurse,site,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,STACH,10900,VA,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
1,AFLT,10900,AF,15654.0,54.0,0,NaN,NaN,0.0,0.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,6,records100/10000/10900_lr,records500/10000/10900_hr
2,SR,8209,SR,12281.0,55.0,0,NaN,NaN,1.0,2.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,10,records100/08000/08209_lr,records500/08000/08209_hr
3,STACH,17620,VA,2007.0,29.0,1,164.0,56.0,7.0,1.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,1,records100/17000/17620_lr,records500/17000/17620_hr
4,SBRAD,12967,VA,8685.0,57.0,0,NaN,NaN,0.0,0.0,...,False,NaN,", I-AVR,",NaN,NaN,NaN,NaN,1,records100/12000/12967_lr,records500/12000/12967_hr


In [3]:

# drop columns
sub_df = df.drop(columns=['diagnosi', 'ecg_id', 'patient_id', 'recording_date', 'report', 'scp_codes', 'infarction_stadium1', 'infarction_stadium2', 'initial_autogenerated_report', 'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'extra_beats', 'filename_lr', 'filename_hr'])
sub_df.head()

,ritmi,age,sex,height,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,VA,54.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,6
1,AF,54.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,6
2,SR,55.0,0,NaN,NaN,1.0,2.0,CS-12,LAD,1.0,False,True,NaN,10
3,VA,29.0,1,164.0,56.0,7.0,1.0,AT-6 C 5.6,NaN,0.0,False,True,NaN,1
4,VA,57.0,0,NaN,NaN,0.0,0.0,CS100 3,MID,NaN,False,False,NaN,1


In [4]:
# fill missing values with mean values for the age, height, weight columns
sub_df['age'].fillna(value=sub_df['age'].mean(), inplace=True)
sub_df['height'].fillna(value=sub_df['height'].mean(), inplace=True)
sub_df['weight'].fillna(value=sub_df['weight'].mean(), inplace=True)

In [5]:
# fill missing values with 0 for these columns
sub_df['nurse'] = sub_df['nurse'].fillna(0)
sub_df['site'] = sub_df['site'].fillna(0)
sub_df['validated_by'] = sub_df['validated_by'].fillna(0)
sub_df['heart_axis'] = sub_df['heart_axis'].fillna('Missing')
sub_df['pacemaker'] = sub_df['pacemaker'].fillna('Missing')

In [6]:
# map categorical values to numerical values
sub_df['ritmi'] = sub_df['ritmi'].map({'SR': 0, 'AF': 1, 'VA': 2}).values
sub_df['second_opinion'] = sub_df['second_opinion'].map({False: 0, True:1}).values
sub_df['validated_by_human'] = sub_df['validated_by_human'].map({False: 0, True:1}).values
sub_df['heart_axis'] = sub_df['heart_axis'].map({'Missing':0, 'LAD':1, 'AXL':2, 'MID':3, 'RAD':4, 'ALAD':5, 'AXR':6}).values
sub_df['pacemaker'] = sub_df['pacemaker'].map({'Missing':0, 'PACE????, nan': 0, '?, nan':0, 'ja, pacemaker':1}).values
sub_df['device'] = sub_df['device'].map({'CS100    3':0, 'CS-12': 1, 'AT-6 C 5.5':2, 'CS-12   E':3, 'AT-6     6':4, 'AT-60    3':5, 'AT-6 C 5.8':6, 'AT-6 C':7, 'AT-6 C 5.0':8, 'AT-6 C 5.3':9, 'AT-6 C 5.6':10}).values

In [7]:
# check sub_df
print(sub_df.shape)
sub_df.head()

(6428, 14)


,ritmi,age,sex,height,weight,nurse,site,device,heart_axis,validated_by,second_opinion,validated_by_human,pacemaker,strat_fold
0,2,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
1,1,54.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,6
2,0,55.0,0,166.796356,69.841845,1.0,2.0,1,1.0,1.0,0,1,0.0,10
3,2,29.0,1,164.000000,56.000000,7.0,1.0,10,0.0,0.0,0,1,0.0,1
4,2,57.0,0,166.796356,69.841845,0.0,0.0,0,3.0,0.0,0,0,0.0,1


In [36]:
# export to csv
sub_df.to_csv('training_13_features.csv', index=False)